# **Demo Script 1**

This is the training portion of our model analysis. To train the model, all of the run 1 data (both fNIRS and fMRI) are loaded. This script will create the model based on the training data, and the model can then be used in the following script (demo 2) in order to determine model performance.

In [1]:
import scipy.io as sio
import pickle

import numpy as np
from scipy.stats import pearsonr
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression

## **Train a model on all of Run 1 Data**

In [3]:
# load fnirs run 1 data
fnirs_file1 = '../data/fnirs/clip1_noUncertain_zhbo_TxRxS.mat'
train_fNIRS = sio.loadmat(fnirs_file1)['zhbo_TxRxS'].transpose((2, 0, 1))  # reshape from TxRxS to SxTxR

# get shape
print(f'train_fNIRS shape is {train_fNIRS.shape}')

# load fMRI run 1 data
fmri_file1 = '../data/fmri/clip1_undenoised_withcartoon_fillmean_TxRxS.mat'
train_fMRI = sio.loadmat(fmri_file1)['bold_TxRxS'].transpose((2, 0, 1))  # reshape from TxRxS to SxTxR
# get shape
print(f'train_fMRI shape is {train_fMRI.shape}')

train_fNIRS shape is (29, 946, 20)
train_fMRI shape is (17, 946, 122)


In [4]:
# Train model
X_num_pcs = 0.9
y_num_pcs = 0.9

# impute within curr_Xt_SxTxR. impute_nan() input shape: RxTxS
train_fNIRS = np.array(
    [
        np.apply_along_axis(lambda x: np.nan_to_num(x, nan=np.nanmean(x)), axis=1, arr=curr_region) \
            for curr_region in train_fNIRS.transpose((2, 1, 0))  # transpose to shape: RxTxS
    ]
).transpose((2, 1, 0))

# stack Xt and yt
train_fNIRS_concat = []
train_fMRI_concat = []

for subj_X in train_fNIRS:
    for subj_y in train_fMRI:
        train_fNIRS_concat.append(subj_X)
        train_fMRI_concat.append(subj_y)

train_fNIRS_concat = np.concatenate(train_fNIRS_concat, axis=0)
train_fMRI_concat = np.concatenate(train_fMRI_concat, axis=0)

# pca for X
X_pca = PCA(n_components=X_num_pcs)
X_train_pc = X_pca.fit_transform(train_fNIRS_concat)

# pca for y
y_pca = PCA(n_components=y_num_pcs)
y_train_pc = y_pca.fit_transform(train_fMRI_concat)

# fit linear reg model on training set
lr_reg = LinearRegression().fit(X=X_train_pc, y=y_train_pc)
# get weights
lr_weights = lr_reg.coef_   # target (fmri pc) * weights on predictors (fnirs pc)

print("Model trained...")

# Save the model
file_path = '../models/sherlock_run1model.pickle'

with open(file_path, 'wb') as f:
  pickle.dump([X_pca, y_pca, lr_reg], f)

Model trained...
